In [26]:
#TO RE-RUN
%reset -f

In [27]:
# coding: utf-8
# In[110]:
from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd
import os

%matplotlib inline

In [31]:
df_grouped=pd.read_csv(os.path.join('resources','diabetic_data_processed_2017_09_29.csv'),';')
print df_grouped.shape

range1=np.array(range(1, 139)).astype(str)
range2=np.array(range(140, 239)).astype(str)
range3=np.array(range(240, 279)).astype(str)
range4=np.array(range(280, 289)).astype(str)
range5=np.array(range(290, 319)).astype(str)
range6=np.array(range(320, 359)).astype(str)
range7=np.array(range(360, 389)).astype(str)
range8=np.array(range(390, 459)).astype(str)
range9=np.array(range(460, 519)).astype(str)
range10=np.array(range(520, 579)).astype(str)
range11=np.array(range(580, 629)).astype(str)
range12=np.array(range(630, 679)).astype(str)
range13=np.array(range(680, 709)).astype(str)
range14=np.array(range(710, 739)).astype(str)
range15=np.array(range(740, 759)).astype(str)
range16=np.array(range(760, 779)).astype(str)
range17=np.array(range(780, 799)).astype(str)
range18=np.array(range(800, 999)).astype(str)

ix = pd.value_counts(df_grouped['diag_1']).index
rangeD2 =  ix[ix.str.contains("250")]
print rangeD2

(101766, 50)
Index([u'250.8', u'250.6', u'250.7', u'250.13', u'250.02', u'250.11',
       u'250.12', u'250.82', u'250.1', u'250.4', u'250', u'250.03', u'250.81',
       u'250.22', u'250.2', u'250.83', u'250.41', u'250.42', u'250.01',
       u'250.92', u'250.23', u'250.43', u'250.3', u'250.33', u'250.93',
       u'250.32', u'250.31', u'250.21', u'250.5', u'250.9', u'250.91',
       u'250.53', u'250.52', u'250.51'],
      dtype='object')


In [32]:
print pd.value_counts(df_grouped['diag_1']).iloc[:5]
print pd.value_counts(df_grouped['diag_2']).iloc[:5]
print pd.value_counts(df_grouped['diag_3']).iloc[:5]

print np.sum(df_grouped["diag_1"].isin(range1))
print np.sum(df_grouped["diag_1"].isin(range2))
print np.sum(df_grouped["diag_1"].isin(range3))
print np.sum(df_grouped["diag_1"].isin(range4))

428    6862
414    6581
786    4016
410    3614
486    3508
Name: diag_1, dtype: int64
276    6752
428    6662
250    6071
427    5036
401    3736
Name: diag_2, dtype: int64
250    11555
401     8289
276     5175
428     4577
427     3955
Name: diag_3, dtype: int64
2768
3400
2935
1084


In [33]:
def remap_disseases(column):
    
    d = []
    col = column
    print column
    
    if col == "Diabetis":
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(rangeD2), df_grouped["diag_2"].isin(rangeD2)),
                df_grouped["diag_3"].isin(rangeD2))].index)
        
    if col == "External causes":
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].str.upper().str.contains("E"), df_grouped["diag_2"].str.upper().str.contains("E")),
                df_grouped["diag_3"].str.upper().str.contains("E"))].index)
        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].str.upper().str.contains("V"), df_grouped["diag_2"].str.upper().str.contains("V")),
                df_grouped["diag_3"].str.upper().str.contains("V"))].index)        
       
    if col == "Infectious and parasitic diseases":      
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range1), df_grouped["diag_2"].isin(range1)),
                df_grouped["diag_3"].isin(range1))].index)

    if col == "Neoplasms":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range2), df_grouped["diag_2"].isin(range2)),
                df_grouped["diag_3"].isin(range2))].index)        

    if col == "Endocrine":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range3), df_grouped["diag_2"].isin(range3)),
                df_grouped["diag_3"].isin(range3))].index)        

    if col == "Blood":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range4), df_grouped["diag_2"].isin(range4)),
                df_grouped["diag_3"].isin(range4))].index)        

    if col == "Mental":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range5), df_grouped["diag_2"].isin(range5)),
                df_grouped["diag_3"].isin(range5))].index)        

    if col == "Nervous":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range6), df_grouped["diag_2"].isin(range6)),
                df_grouped["diag_3"].isin(range6))].index)        

    if col == "Organs":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range7), df_grouped["diag_2"].isin(range7)),
                df_grouped["diag_3"].isin(range7))].index)         

    if col == "Circulatory":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range8), df_grouped["diag_2"].isin(range8)),
                df_grouped["diag_3"].isin(range8))].index)        

    if col == "Respiratory":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range9), df_grouped["diag_2"].isin(range9)),
                df_grouped["diag_3"].isin(range9))].index)        

    if col == "Digestive":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range10), df_grouped["diag_2"].isin(range10)),
                df_grouped["diag_3"].isin(range10))].index)        

    if col == "Genitourinary":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range11), df_grouped["diag_2"].isin(range11)),
                df_grouped["diag_3"].isin(range11))].index) 

    if col == "Pregnancy":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range12), df_grouped["diag_2"].isin(range12)),
                df_grouped["diag_3"].isin(range12))].index)        

    if col == "Skin":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range13), df_grouped["diag_2"].isin(range13)),
                df_grouped["diag_3"].isin(range13))].index)        

    if col == "Muscoskeletal":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range14), df_grouped["diag_2"].isin(range14)),
                df_grouped["diag_3"].isin(range14))].index)        

    if col == "Congenital":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range15), df_grouped["diag_2"].isin(range15)),
                df_grouped["diag_3"].isin(range15))].index) 

    if col == "Perinatal":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range16), df_grouped["diag_2"].isin(range16)),
                df_grouped["diag_3"].isin(range16))].index)        

    if col == "Ill-defined":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range17), df_grouped["diag_2"].isin(range17)),
                df_grouped["diag_3"].isin(range17))].index)        

    if col == "Poisoning":        
        d.extend(
            df_grouped[np.logical_or(
                np.logical_or(df_grouped["diag_1"].isin(range18), df_grouped["diag_2"].isin(range18)),
                df_grouped["diag_3"].isin(range18))].index) 
        
    return [col,d]

In [34]:
from multiprocessing import Pool

pool = Pool()

ix = []
ix = pool.map(remap_disseases,["Diabetis","External causes","Infectious and parasitic diseases",
"Neoplasms","Endocrine","Blood","Mental","Nervous","Organs","Circulatory",
"Respiratory","Digestive","Genitourinary","Pregnancy","Skin",
"Muscoskeletal","Congenital","Perinatal","Ill-defined","Poisoning"])

External causes
Infectious and parasitic diseases
Endocrine
Blood
Diabetis
Circulatory
Mental
Neoplasms
Nervous
Organs
Digestive
Respiratory
Genitourinary
Muscoskeletal
Skin
Pregnancy
Congenital
Perinatal
Ill-defined
Poisoning


In [35]:
for i in range(len(ix)):
    df_grouped[ix[i][0]]=0
    df_grouped[ix[i][0]].ix[ix[i][1]]=1
    
    print ix[i][0]
    print pd.value_counts(df_grouped[ix[i][0]])


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


Diabetis
0    63742
1    38024
Name: Diabetis, dtype: int64
External causes
0    93297
1     8469
Name: External causes, dtype: int64
Infectious and parasitic diseases
0    95437
1     6329
Name: Infectious and parasitic diseases, dtype: int64
Neoplasms
0    95715
1     6051
Name: Neoplasms, dtype: int64
Endocrine
0    67774
1    33992
Name: Endocrine, dtype: int64
Blood
0    95531
1     6235
Name: Blood, dtype: int64
Mental
0    95209
1     6557
Name: Mental, dtype: int64
Nervous
0    98335
1     3431
Name: Nervous, dtype: int64
Organs
0    101148
1       618
Name: Organs, dtype: int64
Circulatory
1    58520
0    43246
Name: Circulatory, dtype: int64
Respiratory
0    79301
1    22465
Name: Respiratory, dtype: int64
Digestive
0    88140
1    13626
Name: Digestive, dtype: int64
Genitourinary
0    84456
1    17310
Name: Genitourinary, dtype: int64
Pregnancy
0    101074
1       692
Name: Pregnancy, dtype: int64
Skin
0    94443
1     7323
Name: Skin, dtype: int64
Muscoskeletal
0    94234
1

#### Save

In [37]:
to_del = ['diag_1','diag_2','diag_3']        
print (to_del)

#Filter_selected cols
filtered_cols = [c for c in df_grouped.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df_grouped[filtered_cols]
print ("df_2",df_2.shape)

df_2.to_csv(os.path.join('resources',"diabetic_data_extended_2017_09_29_14_xrp.csv")) 

['diag_1', 'diag_2', 'diag_3']
('df_2', (101766, 67))
